In [1]:
import string
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
import string 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16514\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16514\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read Dataset

In [2]:
imdb_data = pd.read_csv('./data/IMDB Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### Split the data

In [4]:
train_reviews=imdb_data.review[:30000]
train_sentiments=imdb_data.sentiment[:30000]

val_reviews=imdb_data.review[30000:40000]
val_sentiments=imdb_data.sentiment[30000:40000]

test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]

## Preprocess Dataset

In [5]:
def preprocess(reviews):
    container = []
    for review in reviews:
        review = review.replace("<br />", "")
        for ele in string.punctuation:
                if ele in review:
                        review = review.replace(ele, "")
        container.append(review)
    return container

def preprocess_sentiment(sentiments):
    container = []
    for sentiment in sentiments:
        container.append(sentiment)
    return container

In [6]:
train_reviews = preprocess(train_reviews)
val_reviews = preprocess(val_reviews)
test_reviews = preprocess(test_reviews)
train_sentiments = preprocess_sentiment(train_sentiments)
val_sentiments = preprocess_sentiment(val_sentiments)
test_sentiments = preprocess_sentiment(test_sentiments)

In [7]:
'''
Method that take in the training dataset, then return the positive and negative words log probability.
Input: train_reviews: reviews (sentences) for training
       train_sentiments: sentiments (label) for training
       val_reviews: reviews (sentences) for validation
       test_reviews: reviews (sentences) for testing
       tfidf: boolean variable indicating whether using bow or tfidf
       alpha: laplance smoothing variable, default to be 1.0
       ngram_range: the scale of ngram model will be used, default = (1,1) unigram
return: negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class
        positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class
        mnb: the trained multinomial naive bayes model, later can be used for testing
        transformed_val_reviews: transformed val reviews that later can be used for validation
        transformed_test_reviews: transformed test reviews that later can be used for testing
        vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
'''

def generate_log_prob(train_reviews, train_sentiments,  test_reviews, val_reviews=None, tfidf=False, alpha=1.0, ngram_range = (1,1)):

    if (tfidf):
        #Tfidf vectorizer
        vec=TfidfVectorizer(use_idf=tfidf, ngram_range=ngram_range)
        transformed_train_reviews=vec.fit_transform(train_reviews)
        if val_reviews is not None:
            transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)
    else:
        vec=CountVectorizer(ngram_range=(1,1))
        transformed_train_reviews=vec.fit_transform(train_reviews)
        if val_reviews is not None:
            transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)

    #training the model
    mnb = MultinomialNB(alpha=alpha)

    #fitting the naive bayes for bag of words
    mnb = mnb.fit(transformed_train_reviews, train_sentiments)
    
    negative_log_prob = mnb.feature_log_prob_[0]
    positive_log_prob = mnb.feature_log_prob_[1]

    # Generate two dict: word:log_prob
    negative_word_log_prob_dict = {}
    positive_word_log_prob_dict = {}
    for word, index in vec.vocabulary_.items():
        negative_word_log_prob_dict[word] = negative_log_prob[index]
        positive_word_log_prob_dict[word] = positive_log_prob[index]
    if val_reviews is None:
        return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_test_reviews, vec
    else:
        return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_val_reviews, transformed_test_reviews, vec 

### Bag of Word - Naive Bayes

In [8]:
neg, pos, mnb, transformed_val_reviews, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, test_reviews, val_reviews=val_reviews, alpha = 1)

In [9]:
unseen_neg = mnb.feature_log_prob_[0].min()
unseen_pos = mnb.feature_log_prob_[1].min()
print(unseen_neg, unseen_pos)

-15.032443811652895 -15.049387153559886


In [10]:
val_pred = mnb.predict(transformed_val_reviews)
test_pred = mnb.predict(transformed_test_reviews)
val_acc = accuracy_score(val_sentiments, val_pred)
test_acc = accuracy_score(test_sentiments, test_pred)
print(f"val accuracy is {val_acc}, test accuracy is {test_acc}")


val accuracy is 0.8449, test accuracy is 0.8489


In [11]:
def naive_inference(reweight, pos, neg, unseen_neg, unseen_pos, test_reviews, test_sentiments):
    correct = 0
    for i in range(len(test_reviews)):
        word_list = test_reviews[i].strip().split()
        final_result = 0
        
        for word in word_list:
            weight = 1
            pprob = unseen_pos
            nprob = unseen_neg
            if word in reweight:
                weight = reweight[word]
            if word in pos:
                pprob = pos[word]
            if word in neg:
                nprob = neg[word]
            final_result += weight*pprob - weight*nprob
        if (final_result > 0 and test_sentiments[i] == "positive") or (final_result < 0 and test_sentiments[i] == "negative"):
            correct += 1
    print(correct/len(test_reviews))
    return correct/len(test_reviews)

In [12]:
val_pred = naive_inference({}, pos, neg, unseen_neg, unseen_pos, val_reviews, val_sentiments)

0.8287


## Reweight the model using perceptron

In [13]:
def perceptron_update(positive_prob_dict, negative_prob_dict, unseen_pos, unseen_neg, val_reviews, val_sentiments, test_reviews, test_sentiments, max_iter=10, learning_rate=0.02):
    """
    need to handle unseen word here
    has to be a method
    """
    initial_weight = {}
    for i in range(max_iter):
        random_array = np.arange(len(val_sentiments))
        np.random.shuffle(random_array)
        correct = 0
        for index in random_array:
            word_list = val_reviews[index].strip().split()
            final_result = 0
            for word in word_list:
                weight = 1
                pprob = unseen_pos
                nprob = unseen_neg
                if word not in initial_weight:
                    initial_weight[word] = 1
                if word in initial_weight:
                    weight = initial_weight[word]
                if word in positive_prob_dict:
                    pprob = positive_prob_dict[word]
                if word in negative_prob_dict:
                    nprob = negative_prob_dict[word]
                final_result += weight*pprob - weight*nprob
            if (final_result > 0 and val_sentiments[index] == "positive") or (final_result < 0 and val_sentiments[index] == "negative"):
                correct += 1
                for word in word_list:
                    initial_weight[word] -= learning_rate/len(val_sentiments)*initial_weight[word]
            else:
                for word in word_list:
                    pprob = unseen_pos
                    nprob = unseen_neg
                    if word in positive_prob_dict:
                        pprob = positive_prob_dict[word]
                    if word in negative_prob_dict:
                        nprob = negative_prob_dict[word]
                    if val_sentiments[index] == "positive":
                        initial_weight[word] += learning_rate*((pprob-nprob) - 1/len(val_sentiments)*initial_weight[word])
                    else:
                        initial_weight[word] += learning_rate*((nprob-pprob) - 1/len(val_sentiments)*initial_weight[word])
        test_acc = naive_inference(initial_weight, positive_prob_dict, negative_prob_dict, unseen_neg, unseen_pos, test_reviews, test_sentiments)
        print(f"finish training epoch {i}, the val accuracy is {correct/len(val_sentiments)}, the test accuracy is {test_acc}")
        
    return initial_weight

In [14]:
reweight = perceptron_update(pos, neg, unseen_pos, unseen_neg, val_reviews, val_sentiments, test_reviews, test_sentiments)

0.8689
finish training epoch 0, the val accuracy is 0.8582, the test accuracy is 0.8689
0.8746
finish training epoch 1, the val accuracy is 0.8773, the test accuracy is 0.8746
0.8768
finish training epoch 2, the val accuracy is 0.887, the test accuracy is 0.8768
0.8755
finish training epoch 3, the val accuracy is 0.8927, the test accuracy is 0.8755
0.8765
finish training epoch 4, the val accuracy is 0.8965, the test accuracy is 0.8765
0.8742
finish training epoch 5, the val accuracy is 0.9032, the test accuracy is 0.8742
0.8725
finish training epoch 6, the val accuracy is 0.9055, the test accuracy is 0.8725
0.8746
finish training epoch 7, the val accuracy is 0.9112, the test accuracy is 0.8746
0.8694
finish training epoch 8, the val accuracy is 0.9189, the test accuracy is 0.8694
0.8733
finish training epoch 9, the val accuracy is 0.9212, the test accuracy is 0.8733


In [15]:
weight_printed_version = dict(sorted(reweight.items(), key=lambda item: item[1]))
with open("weight_dict.txt", "w") as f:
    for k, v in weight_printed_version.items():
        f.write(str(k))
        f.write(": ")
        f.write(str(v))
        f.write('\n')


In [16]:
import pickle

with open('weight.pickle', 'wb') as handle:
    pickle.dump(reweight, handle, protocol=pickle.HIGHEST_PROTOCOL)